In [1]:
import pandas as pd
import difflib

In [50]:
data = pd.read_csv('TEST__dump_parsed.tsv', sep='\t')

In [102]:
test_data = data[data.title == data.title.unique()[0]]
# test_data = CleanTextData.clean_and_filter(test_data,text_col='text')

In [116]:
import CleanTextData
def clean_text(text):
    try:
        text = CleanTextData.remove_date(text)
        text = CleanTextData.substitute_patterns(text, CleanTextData.pre_sub_patterns)
        text = CleanTextData.strip_mw(text)
        text = CleanTextData.strip_html(text)
        text = CleanTextData.substitute_patterns(text, CleanTextData.post_sub_patterns)
    except TypeError:
        # the string is empty
        text = ''
        
    return text

def diff_clean_text(data):
    Differ = difflib.Differ()
    ADDED = []
    ADDED_bytes = []
    DELETED = []
    DELETED_bytes = []
    for row in data.iterrows():
        idx = row[0]
        content = row[1]
        assert 'title' in content.keys()
        assert 'text' in content.keys()

        title = content['title']
        cleaned_text = clean_text(content['text'])


        try:
            if(title == title_prev):
                diff = list(Differ.compare(
                            cleaned_text_prev.split(), 
                            cleaned_text.split()
                            ))

                add_words = []
                del_words = []
                for d in diff:
                    if(d[0] == '+'):
                        add_words.append(d[2:])
                    elif(d[0] == '-'):
                        del_words.append(d[2:])
                added = ' '.join(add_words)
                deleted = ' '.join(del_words)

            else:
                # at the beginning of new page
                added = cleaned_text
                deleted = ''

        except NameError:
            # at the beginning of the file
            added = cleaned_text
            deleted = ''

        ADDED.append(added)
        ADDED_bytes.append(len(added))
        DELETED.append(deleted)
        DELETED_bytes.append(len(deleted))

        title_prev = title
        cleaned_text_prev = cleaned_text
        
    data['Added'] = ADDED
    data['Added_Bytes'] = ADDED_bytes
    data['Deleted'] = DELETED
    data['Deleted_Bytes'] = DELETED_bytes
    
    return data


In [117]:
%time new_data = diff_clean_text(data)

CPU times: user 16min 21s, sys: 172 ms, total: 16min 22s
Wall time: 16min 21s


In [150]:
test_idx = 1111
print('\n'.join(Differ.compare(
            new_data.text[test_idx - 1].splitlines(), 
            new_data.text[test_idx].splitlines()
       )))

  Is this article a stub?
  
  
  
  
  Optical disc can be loaded with tray or slot-in thingymabob.
+ 
+ In my Computer Information Systems class we were told that there are two types of optical disks, the CD and the DVD. Now I'm reading this article and I'm like what the hell? Not that I trust my class over wikipedia. Obviously. [[User:Erik E.|Erik E.]] 02:20, 13 February 2007 (UTC)


In [152]:
print('ADDED\n', new_data.Added[test_idx])
print('\nDELETED\n', new_data.Deleted[test_idx])

ADDED
 In my Computer Information Systems class we were told that there are two types of optical disks, the CD and the DVD. Now I'm reading this article and I'm like what the hell? Not that I trust my class over wikipedia. Obviously.

DELETED
 
